# Augment data for model segment 

In [1]:
from ultralytics import YOLO
import os
import torch
from matplotlib import pyplot as plt
import json
import numpy as np
import random
import cv2
from copy import deepcopy
from glob import glob
from tqdm import tqdm
from typing import Iterable, Optional, Tuple

In [3]:
print("CUDA available:", torch.cuda.is_available())
print("CUDA device count:", torch.cuda.device_count())
if torch.cuda.is_available():
    print("Current CUDA device:", torch.cuda.current_device())
    print("CUDA device name:", torch.cuda.get_device_name(torch.cuda.current_device()))

CUDA available: True
CUDA device count: 1
Current CUDA device: 0
CUDA device name: NVIDIA GeForce GTX 1650


## Adding playing_field mask into lable json file funtion:


In [ ]:
def add_playing_field_mask_to_json(img_path, json_path):
    """
    Function: add_playing_field_mask_to_json

    Purpose:
        - Run a YOLO segmentation model on the input image to detect the playing field.
        - Append each detected polygon mask as a LabelMe "shape" with label "playing_field".
        - Overwrite the original LabelMe JSON file in place.

    Args:
        img_path (str): Path to the input image file.
        json_path (str): Path to the LabelMe JSON file corresponding to the image.

    Outputs:
        - The JSON file at `json_path` is updated on disk (no value returned).

    Notes:
        - Expects a YOLO segmentation model at "Model_playing_field/best.pt".
        - Detected masks are appended as "polygon" shapes to the "shapes" array.
        - Coordinates are stored as float pairs (x, y) in LabelMe format.
    """
    # Load YOLO model
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    model_path = r"Model_playing_field\best.pt"
    print(f"[INFO] Loading YOLO model from: {model_path} (device: {device})")
    model = YOLO(model_path).to(device)

    # Run inference
    print(f"[INFO] Running inference on: {img_path}")
    results = model(img_path)

    # Load original LabelMe JSON
    with open(json_path, 'r', encoding='utf-8') as f:
        labelme_data = json.load(f)

    # Ensure "shapes" field exists
    labelme_data.setdefault("shapes", [])

    # Append detected polygon masks into "shapes"
    added_count = 0
    for result in results:
        if result.masks is None:
            continue
        for mask_xy in result.masks.xy:  # list of Nx2 arrays
            coords = mask_xy.tolist()
            shape = {
                "label": "playing_field",
                "points": coords,
                "group_id": None,        # leave as null
                "shape_type": "polygon",
                "flags": {},
                "mask": None             # leave as null
            }
            labelme_data["shapes"].append(shape)
            added_count += 1

    # Write back to JSON
    with open(json_path, 'w', encoding='utf-8') as f:
        json.dump(labelme_data, f, indent=2, ensure_ascii=False)

    print(f"[OK] Updated JSON: {json_path} | Added {added_count} 'playing_field' polygon(s).")


# Function to select 1 point without overlapping the mask:


In [ ]:
import json
import random
from typing import Iterable, Optional, Tuple
import cv2
import numpy as np

def get_random_point_outside_masks(
    image_path: str,
    labelme_json_path: str,
    exclude_labels: Optional[Iterable[str]] = None
) -> Optional[Tuple[int, int]]:
    """
    Select a random image coordinate that lies outside all polygon masks
    defined in a LabelMe JSON file.

    Args:
        image_path (str):
            Path to the source image.
        labelme_json_path (str):
            Path to the corresponding LabelMe JSON file.
        exclude_labels (Iterable[str] | None, optional):
            Labels to be excluded from masking (i.e., polygons with these labels
            will NOT be filled into the mask). If None, all polygons are masked.

    Returns:
        Optional[Tuple[int, int]]:
            A tuple (x, y) for a valid point outside the masks,
            or None if no valid point exists.

    Raises:
        FileNotFoundError:
            If the image cannot be read from `image_path`.
        json.JSONDecodeError:
            If the JSON file is malformed.

    Notes:
        - Polygon coordinates in LabelMe are floats; they are rounded to the
          nearest integer pixel before rasterization.
        - A polygon must have at least 3 points to be considered valid.
    """
    # Load image
    image = cv2.imread(image_path)
    if image is None:
        raise FileNotFoundError(f"Cannot read image from: {image_path}")
    height, width = image.shape[:2]

    # Binary mask where 1 indicates masked (occupied) pixels
    mask = np.zeros((height, width), dtype=np.uint8)

    # Load LabelMe JSON
    with open(labelme_json_path, "r", encoding="utf-8") as f:
        data = json.load(f)

    # Build exclusion set (labels not to be masked)
    exclude_set = set(exclude_labels) if exclude_labels is not None else None

    # Rasterize polygons into the mask
    for shape in data.get("shapes", []):
        label = shape.get("label")
        points = shape.get("points")

        if not points:
            continue

        # Mask rule:
        # - If exclude_labels is None  -> mask all polygons.
        # - Else, mask only polygons whose label is NOT in exclude_set.
        if exclude_set is not None and label in exclude_set:
            continue

        polygon = np.asarray(points, dtype=np.float32)
        if polygon.ndim != 2 or polygon.shape[0] < 3:
            continue  # invalid polygon

        # Round to integer pixel coordinates for OpenCV fill
        polygon = np.rint(polygon).astype(np.int32)
        cv2.fillPoly(mask, [polygon], color=1)

    # Find candidate points outside all masks (mask == 0)
    ys, xs = np.where(mask == 0)
    if xs.size == 0:
        return None

    # Choose one at random
    idx = random.randrange(xs.size)
    return int(xs[idx]), int(ys[idx])


## Function to insert aug_logo image into original image and update mask of aug_logo into json file:

In [ ]:
def paste_multiple_aug_logos(
    org_img: str,
    org_img_json: str,
    logo_aug_folder: str,
    output_dir: str,
    max_attempts: int = 20
) -> None:
    """
    Function: paste_multiple_aug_logos

    Purpose:
        - Paste multiple augmented logo images onto a background image without overlapping
          existing polygons/shapes (using a binary occupancy mask).
        - Update the corresponding LabelMe JSON by appending transformed polygons from
          the pasted logos.
        - Save the merged image and updated JSON to the output directory.

    Args:
        org_img (str):
            Path to the background image.
        org_img_json (str):
            Path to the LabelMe JSON of the background image.
        logo_aug_folder (str):
            Directory containing augmented logo images and their LabelMe JSONs
            (each logo image must have a matching .json with "shapes").
        output_dir (str):
            Output directory to write the merged image and JSON.
        max_attempts (int, optional):
            Maximum number of random placement trials per logo. Defaults to 20.

    Outputs:
        - Writes "<background_stem>_merged.jpg" and "<background_stem>_merged.json"
          to `output_dir`.

    Notes:
        - This function expects a helper `get_random_point_outside_masks(image_path, json_path, exclude_labels=None)`
          that returns a random (x, y) coordinate outside all polygons in the background JSON.
        - Logo images larger than 10% of the background width/height are downscaled to fit.
        - Occupancy is enforced by a binary mask (rectangle area of the pasted logo),
          preventing overlaps with existing polygons or previously pasted logos.
    """
    os.makedirs(output_dir, exist_ok=True)

    # Load background image
    bg = cv2.imread(org_img)
    if bg is None:
        raise FileNotFoundError(f"[ERROR] Cannot read background image: {org_img}")
    h_bg, w_bg = bg.shape[:2]

    # Load background LabelMe JSON
    with open(org_img_json, 'r', encoding='utf-8') as f:
        bg_json = json.load(f)
    bg_json.setdefault("shapes", [])

    # Build a binary occupancy mask from existing polygons
    mask_block = np.zeros((h_bg, w_bg), dtype=np.uint8)
    for shape in bg_json.get("shapes", []):
        if "points" in shape and shape["points"]:
            pts = np.array(shape["points"], dtype=np.int32)
            cv2.fillPoly(mask_block, [pts], 1)

    # Collect logo image files
    logo_files = [
        f for f in os.listdir(logo_aug_folder)
        if f.lower().endswith(('.jpg', '.jpeg', '.png'))
    ]
    if not logo_files:
        print("[WARN] No logo images found in the augmentation folder.")
        return

    # Randomly select a subset of logos (between 3 and 7)
    selected_logos = random.sample(logo_files, k=min(len(logo_files), random.randint(3, 7)))

    for logo_file in selected_logos:
        base_name = os.path.splitext(logo_file)[0]
        aug_logo = os.path.join(logo_aug_folder, logo_file)
        aug_logo_json = os.path.join(logo_aug_folder, base_name + ".json")

        if not os.path.exists(aug_logo_json):
            print(f"[WARN] Missing JSON for logo: {logo_file} — skipped.")
            continue

        # Load logo image
        logo = cv2.imread(aug_logo)
        if logo is None:
            print(f"[WARN] Cannot read logo image: {aug_logo} — skipped.")
            continue
        h_logo_orig, w_logo_orig = logo.shape[:2]

        # Resize logo if it exceeds 10% of background dimensions
        max_ratio = 0.1
        scale_x = scale_y = 1.0
        if h_logo_orig > h_bg * max_ratio or w_logo_orig > w_bg * max_ratio:
            scale = min((w_bg * max_ratio) / w_logo_orig, (h_bg * max_ratio) / h_logo_orig)
            new_w = int(w_logo_orig * scale)
            new_h = int(h_logo_orig * scale)
            logo = cv2.resize(logo, (new_w, new_h))
            scale_x = new_w / w_logo_orig
            scale_y = new_h / h_logo_orig
        else:
            new_w, new_h = w_logo_orig, h_logo_orig

        # Load logo polygons from its LabelMe JSON
        with open(aug_logo_json, 'r', encoding='utf-8') as f:
            logo_shapes = json.load(f).get("shapes", [])

        # Search for a valid placement (within bounds, no overlap with occupancy mask)
        valid_position = None
        for _ in range(max_attempts):
            point = get_random_point_outside_masks(org_img, org_img_json)
            if point is None:
                continue
            x_offset, y_offset = point
            if x_offset + new_w > w_bg or y_offset + new_h > h_bg:
                continue
            roi = mask_block[y_offset:y_offset + new_h, x_offset:x_offset + new_w]
            if np.any(roi > 0):
                continue
            valid_position = (x_offset, y_offset)
            break

        if valid_position is None:
            print(f"[SKIP] No valid placement found for {logo_file} — skipped.")
            continue

        # Update background image (paste logo)
        x_offset, y_offset = valid_position
        bg[y_offset:y_offset + new_h, x_offset:x_offset + new_w] = logo

        # Update occupancy mask (rectangle area of pasted logo)
        mask_block[y_offset:y_offset + new_h, x_offset:x_offset + new_w] = 1

        # Append transformed polygons to background JSON
        for shape in logo_shapes:
            if "points" not in shape or not shape["points"]:
                continue
            new_shape = deepcopy(shape)
            new_shape["points"] = [
                [pt[0] * scale_x + x_offset, pt[1] * scale_y + y_offset]
                for pt in shape["points"]
            ]
            bg_json["shapes"].append(new_shape)

    # === Save results ===
    base_img_name = os.path.splitext(os.path.basename(org_img))[0]
    output_img_name = base_img_name + "_merged.jpg"
    output_json_name = base_img_name + "_merged.json"

    output_img_path = os.path.join(output_dir, output_img_name)
    output_json_path = os.path.join(output_dir, output_json_name)

    # Update LabelMe metadata
    bg_json["imagePath"] = output_img_name
    bg_json["imageData"] = None
    bg_json["imageHeight"] = h_bg
    bg_json["imageWidth"] = w_bg

    cv2.imwrite(output_img_path, bg)
    with open(output_json_path, 'w', encoding='utf-8') as f:
        json.dump(bg_json, f, indent=2, ensure_ascii=False)

    print(f"[OK] Wrote merged image and JSON to: {output_dir}")
    print(f"     - Image: {output_img_name}")
    print(f"     - JSON : {output_json_name}")


## Function to delete playing_field masks in a folder:

In [3]:
def remove_playing_field_masks_from_folder(folder_path: str) -> None:
    """
    Function: remove_playing_field_masks_from_folder

    Purpose:
        - Scan a directory for LabelMe JSON files and remove all shapes
          whose label is "playing_field".
        - Overwrite JSON files in place when changes occur.

    Args:
        folder_path (str): Path to the directory containing LabelMe JSON files.

    Outputs:
        - Console messages indicating per-file removals and a final total.

    Notes:
        - Files without a "shapes" array or without matching labels are left unchanged.
    """
    json_paths = glob(os.path.join(folder_path, "*.json"))

    if not json_paths:
        print(f"[INFO] No JSON files found in: {folder_path}")
        return

    total_removed = 0

    for json_path in json_paths:
        try:
            with open(json_path, "r", encoding="utf-8") as f:
                data = json.load(f)
        except Exception as e:
            print(f"[ERROR] Failed to read {json_path}: {e}")
            continue

        shapes = data.get("shapes")
        if not isinstance(shapes, list):
            print(f"[INFO] No 'shapes' array in: {json_path} — skipped.")
            continue

        original_count = len(shapes)
        data["shapes"] = [s for s in shapes if s.get("label") != "playing_field"]
        removed_count = original_count - len(data["shapes"])

        if removed_count > 0:
            try:
                with open(json_path, "w", encoding="utf-8") as f:
                    json.dump(data, f, indent=2, ensure_ascii=False)
                print(f"[OK] Removed {removed_count} 'playing_field' mask(s) in: {json_path}")
                total_removed += removed_count
            except Exception as e:
                print(f"[ERROR] Failed to write {json_path}: {e}")
        else:
            print(f"[INFO] No 'playing_field' masks found in: {json_path}")

    print(f"[DONE] Total masks removed: {total_removed}")


# -------------------Main code augment---------------------

In [ ]:
"""
Script: run_augmentation_pipeline.py

Purpose:
    - Iterate through original images and ensure each has a corresponding LabelMe JSON.
    - For each image:
        1) Add a "playing_field" mask via `add_playing_field_mask_to_json`.
        2) Paste multiple augmented logos via `paste_multiple_aug_logos`.
    - Display progress using a tqdm progress bar.

Inputs:
    - org_img_folder:  Directory containing source images (.jpg/.jpeg/.png) and their JSONs.
    - logo_aug_folder: Directory containing augmented logos and their LabelMe JSONs.
    - output_folder:   Destination directory for merged outputs.

Outputs:
    - Merged images and updated JSON files saved to `output_folder`.

Notes:
    - Assumes the helper functions `add_playing_field_mask_to_json` and
      `paste_multiple_aug_logos` are defined/imported elsewhere.
"""

# === CONFIGURATION ===
org_img_folder = r"data_json"
logo_aug_folder = r"logobet_augment\aug_logo"
output_folder   = r"data_aug_json"

# === COLLECT VALID IMAGE FILES ===
org_img_files = [
    f for f in os.listdir(org_img_folder)
    if f.lower().endswith(('.jpg', '.jpeg', '.png'))
]

# === PROCESS IMAGES WITH A PROGRESS BAR ===
for org_img in tqdm(org_img_files, desc="Processing images"):
    org_img_path = os.path.join(org_img_folder, org_img)

    # Find the corresponding JSON (same stem)
    org_json_path = os.path.splitext(org_img_path)[0] + '.json'
    if not os.path.exists(org_json_path):
        print(f"[WARN] LabelMe JSON not found for image: {org_img_path}")
        continue

    # Step 1: Add playing_field masks
    add_playing_field_mask_to_json(org_img_path, org_json_path)

    # Step 2: Paste multiple augmented logos and update JSON
    try:
        paste_multiple_aug_logos(
            org_img_path,
            org_json_path,
            logo_aug_folder,
            output_folder
        )
    except Exception as e:
        print(f"[ERROR] Failed while pasting logos for '{org_img}': {e}")


Processing images:   0%|          | 0/2041 [00:00<?, ?it/s]


image 1/1 d:\Code\EveMask\Augment_data\data_json\BostonCeltics_NewYorkKnicks_1205_Chaudpb_000001.jpg: 384x640 2 playing_fields, 63.0ms
Speed: 1.6ms preprocess, 63.0ms inference, 5.1ms postprocess per image at shape (1, 3, 384, 640)


Processing images:   0%|          | 1/2041 [00:01<50:30,  1.49s/it]


image 1/1 d:\Code\EveMask\Augment_data\data_json\BostonCeltics_NewYorkKnicks_1205_Chaudpb_000002.jpg: 384x640 2 playing_fields, 51.3ms
Speed: 1.7ms preprocess, 51.3ms inference, 2.7ms postprocess per image at shape (1, 3, 384, 640)


Processing images:   0%|          | 2/2041 [00:02<42:51,  1.26s/it]

...

Processing images: 100%|█████████▉| 2039/2041 [39:12<00:02,  1.10s/it]


image 1/1 d:\Code\EveMask\Augment_data\data_json\What_A_Move_1705_khaihoan2003_000758.jpg: 384x640 2 playing_fields, 51.0ms
Speed: 1.7ms preprocess, 51.0ms inference, 2.4ms postprocess per image at shape (1, 3, 384, 640)


Processing images: 100%|█████████▉| 2040/2041 [39:13<00:01,  1.16s/it]


image 1/1 d:\Code\EveMask\Augment_data\data_json\What_A_Move_1705_khaihoan2003_000759.jpg: 384x640 (no detections), 51.3ms
Speed: 1.8ms preprocess, 51.3ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)


Processing images: 100%|██████████| 2041/2041 [39:15<00:00,  1.15s/it]


## Delete playing_field mask in Json files as needed:

In [ ]:
org_img_folder = r"data_aug_json"

remove_playing_field_masks_from_folder(org_img_folder)

[INFO] No 'playing_field' masks found in: data_aug_json\Aug_BostonCeltics_NewYorkKnicks_1205_Chaudpb_000001.json
[INFO] No 'playing_field' masks found in: data_aug_json\Aug_BostonCeltics_NewYorkKnicks_1205_Chaudpb_000002.json
[INFO] No 'playing_field' masks found in: data_aug_json\Aug_BostonCeltics_NewYorkKnicks_1205_Chaudpb_000003.json
[INFO] No 'playing_field' masks found in: data_aug_json\Aug_BostonCeltics_NewYorkKnicks_1205_Chaudpb_000004.json
[INFO] No 'playing_field' masks found in: data_aug_json\Aug_BostonCeltics_NewYorkKnicks_1205_Chaudpb_000006.json
[INFO] No 'playing_field' masks found in: data_aug_json\Aug_BostonCeltics_NewYorkKnicks_1205_Chaudpb_000007.json
[INFO] No 'playing_field' masks found in: data_aug_json\Aug_BostonCeltics_NewYorkKnicks_1205_Chaudpb_000008.json
[INFO] No 'playing_field' masks found in: data_aug_json\Aug_BostonCeltics_NewYorkKnicks_1205_Chaudpb_000009.json
[INFO] No 'playing_field' masks found in: data_aug_json\Aug_BostonCeltics_NewYorkKnicks_1205_Cha